In [2]:
import tensorflow as tf
from tensorflow import keras
from keras import datasets, layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import precision_score, recall_score
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import random
import os
import shutil

2024-01-03 15:24:53.750905: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-03 15:24:53.891717: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-03 15:24:53.891765: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-03 15:24:53.917197: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-03 15:24:54.001199: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-03 15:24:54.002036: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [1]:
dataset_dir = '/workspaces/LungsXray-FP/data/raw/BINARY_PNEUMONIA'

In [3]:
image_size = (150, 150)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


Found 1138 images belonging to 2 classes.
Found 283 images belonging to 2 classes.


In [4]:
num_classes = len(train_generator.class_indices)

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='sigmoid')  # Clasificación multiclase, usar 'softmax'
])

model.compile(optimizer='adam',
              loss='mean_absolute_error',
              metrics=['accuracy'])

2024-01-03 15:25:31.064031: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 18939904 exceeds 10% of free system memory.
2024-01-03 15:25:31.092574: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 18939904 exceeds 10% of free system memory.
2024-01-03 15:25:31.098490: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 18939904 exceeds 10% of free system memory.


In [5]:
epochs = 10

history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

Epoch 1/10


2024-01-03 15:25:47.683047: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 18939904 exceeds 10% of free system memory.
2024-01-03 15:25:47.686477: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 18939904 exceeds 10% of free system memory.


36/36 [==============================] - 75s 2s/step - loss: 0.1763 - accuracy: 0.8155 - val_loss: 0.1625 - val_accuracy: 0.8375
Epoch 2/10
36/36 [==============================] - 72s 2s/step - loss: 0.1652 - accuracy: 0.8348 - val_loss: 0.1625 - val_accuracy: 0.8375
Epoch 3/10
36/36 [==============================] - 70s 2s/step - loss: 0.1652 - accuracy: 0.8348 - val_loss: 0.1625 - val_accuracy: 0.8375
Epoch 4/10
36/36 [==============================] - 71s 2s/step - loss: 0.1652 - accuracy: 0.8348 - val_loss: 0.1625 - val_accuracy: 0.8375
Epoch 5/10
36/36 [==============================] - 72s 2s/step - loss: 0.1652 - accuracy: 0.8348 - val_loss: 0.1625 - val_accuracy: 0.8375
Epoch 6/10
36/36 [==============================] - 75s 2s/step - loss: 0.1652 - accuracy: 0.8348 - val_loss: 0.1625 - val_accuracy: 0.8375
Epoch 7/10
36/36 [==============================] - 70s 2s/step - loss: 0.1652 - accuracy: 0.8348 - val_loss: 0.1625 - val_accuracy: 0.8375
Epoch 8/10
36/36 [=============

In [6]:
test_loss, test_acc = model.evaluate(validation_generator, verbose=2)
print('\nExactitud en el conjunto de validación:', test_acc)


9/9 - 7s - loss: 0.1625 - accuracy: 0.8375 - 7s/epoch - 727ms/step

Exactitud en el conjunto de validación: 0.8374558091163635


In [ ]:
model.save('modelo_binary_PNEUMONIA.keras')

In [ ]:
model = load_model('modelo_binary_PNEUMONIA.keras')  # Ajusta el nombre de tu modelo
# Preprocesamiento de datos para el conjunto de prueba
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    dataset_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',  # Ajusta si tu modelo espera etiquetas categóricas
    shuffle=False
)

In [ ]:
# Obtener las predicciones del modelo para el conjunto de prueba
predictions = model.predict(test_generator)
# Convertir las predicciones a clases (si es necesario)
predicted_classes = np.argmax(predictions, axis=1)
# Obtener las etiquetas verdaderas
true_classes = test_generator.classes
# Calcular la precisión y el recall por clase
classification_rep = classification_report(true_classes, predicted_classes)
accuracy = accuracy_score(true_classes, predicted_classes)
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("Informe de clasificación:")
print(classification_rep)
print("Exactitud general:")
print(accuracy)
print("Matriz de confusión:")
print(conf_matrix)